# Analysis: Bakayoko replacement

Analysis: Bakayoko replacement \
Name: Yasin Tunçbilek \
Date: 9 February 2024

## Packages

Library to send HTTP requests

In [1]:
import requests

Library to parse HTML

In [2]:
from bs4 import BeautifulSoup

Library to deal with various types of I/O. Needed because read_html gives an error.

In [3]:
from io import StringIO

Pandas library for data analysis and manipulation

In [4]:
import pandas as pd

## Player stats

### URLs to stats

URL of a competition where stats reside in

In [5]:
competition_url = "https://fbref.com/en/comps/23/2023-2024/stats/2023-2024-Eredivisie-Stats"

Download HTML of competition page

In [6]:
data = requests.get(competition_url)

Initialise library with downloaded HTML

In [7]:
soup = BeautifulSoup(data.text)

Select menu where URLs to various stats reside in

In [8]:
menu_stats = soup.select('li.full.hasmore')[0]

Find all tags where URLs to various stats reside in

In [9]:
urls_stats = menu_stats.find_all('a')

Get href property of each link

In [10]:
urls_stats = [u.get("href") for u in urls_stats]

Make full URLs

In [11]:
urls_stats = [f"https://fbref.com{l}" for l in urls_stats]

Print list of URLS where various stats reside in

In [12]:
urls_stats

['https://fbref.com/en/comps/23/2023-2024/stats/2023-2024-Eredivisie-Stats',
 'https://fbref.com/en/comps/23/2023-2024/keepers/2023-2024-Eredivisie-Stats',
 'https://fbref.com/en/comps/23/2023-2024/keepersadv/2023-2024-Eredivisie-Stats',
 'https://fbref.com/en/comps/23/2023-2024/shooting/2023-2024-Eredivisie-Stats',
 'https://fbref.com/en/comps/23/2023-2024/passing/2023-2024-Eredivisie-Stats',
 'https://fbref.com/en/comps/23/2023-2024/passing_types/2023-2024-Eredivisie-Stats',
 'https://fbref.com/en/comps/23/2023-2024/gca/2023-2024-Eredivisie-Stats',
 'https://fbref.com/en/comps/23/2023-2024/defense/2023-2024-Eredivisie-Stats',
 'https://fbref.com/en/comps/23/2023-2024/possession/2023-2024-Eredivisie-Stats',
 'https://fbref.com/en/comps/23/2023-2024/playingtime/2023-2024-Eredivisie-Stats',
 'https://fbref.com/en/comps/23/2023-2024/misc/2023-2024-Eredivisie-Stats']

### Standard stats

URL of standard stats

In [13]:
standard_url = urls_stats[0]

Download HTML of standard stats page. Player table is commented out and therefore not retrievable. Therefore a replace.

In [14]:
players_standard_data = requests.get(standard_url).text.replace('<!--','').replace('-->','')

Wrap in a 'StringIO' object to read from a literal string

In [15]:
players_standard_data = StringIO(players_standard_data)

Create dataframe of players standard stats

In [16]:
players_standard_stats = pd.read_html(players_standard_data, attrs = {'id':'stats_standard'})[0]

Drop the first level of the dataframe

In [17]:
players_standard_stats.columns = players_standard_stats.columns.droplevel()

Drop rows where column names occur because of the 'folded' table

In [18]:
players_standard_stats = players_standard_stats.loc[players_standard_stats['Player'] != 'Player']

Keep only relevant columns

In [19]:
players_standard_stats = players_standard_stats[["Player", "Nation", "Pos", "Squad", "Age", "Born", "Min", "90s"]]

Keep last three characters of values in Nation column 

In [20]:
players_standard_stats["Nation"] = players_standard_stats["Nation"].str[-3:]

Make column names with lowercase

In [21]:
players_standard_stats.columns = players_standard_stats.columns.str.lower()

Print first rows of dataframe

In [22]:
print(players_standard_stats)

                    player nation    pos          squad age  born   min   90s
0      Patrick van Aanholt    NED     DF  PSV Eindhoven  32  1990  1114  12.4
1          Paxten Aaronson    USA     MF        Vitesse  19  2003  1253  13.9
2             Jayden Addai    NED     FW     AZ Alkmaar  17  2005   297   3.3
3           Bobby Adekanye    NED     FW   Go Ahead Eag  24  1999  1804  20.0
4            Shawn Adewoye    BEL     DF   RKC Waalwijk  23  2000  2338  26.0
..                     ...    ...    ...            ...  ..   ...   ...   ...
539      Lequincio Zeefuik    NED  MF,FW     AZ Alkmaar  18  2004    59   0.7
540      Lequincio Zeefuik    NED  FW,MF       Volendam  18  2004   773   8.6
541         Ramiz Zerrouki    ALG     MF      Feyenoord  25  1998  1387  15.4
542      Giovanni van Zwam    NED     DF        Vitesse  19  2004   492   5.5
543  Willum Þór Willumsson    ISL     MF   Go Ahead Eag  24  1998  2672  29.7

[524 rows x 8 columns]


In [23]:
players_standard_stats.shape

(524, 8)

### Shooting stats

URL of shooting stats

In [46]:
shooting_url = urls_stats[3]

Download HTML of shooting page. Table is commented out. Therefore a replace.

In [47]:
players_shooting_data = requests.get(shooting_url).text.replace('<!--','').replace('-->','')

Wrap in a 'StringIO' object to read from a literal string

In [48]:
players_shooting_data = StringIO(players_shooting_data)

Create dataframe of players shooting stats

In [49]:
players_shooting_stats = pd.read_html(players_shooting_data, attrs = {'id':'stats_shooting'})[0]

Drop the first level of the dataframe

In [50]:
players_shooting_stats.columns = players_shooting_stats.columns.droplevel()

Drop rows where column names occur because of the 'folded' table

In [51]:
players_shooting_stats = players_shooting_stats.loc[players_shooting_stats['Player'] != 'Player']

Keep only relevant columns

In [64]:
players_shooting_stats = players_shooting_stats[["Player", "Squad", "Age", "Sh", "npxG", "npxG/Sh"]]

Print first rows of dataframe

In [53]:
players_shooting_stats.head()

,Player,Squad,Age,Sh,npxG,npxG/Sh
0,Patrick van Aanholt,PSV Eindhoven,32,26,2.1,0.08
1,Paxten Aaronson,Vitesse,19,38,5.0,0.14
2,Jayden Addai,AZ Alkmaar,17,7,0.2,0.03
3,Bobby Adekanye,Go Ahead Eag,24,40,2.6,0.06
4,Shawn Adewoye,RKC Waalwijk,23,16,1.3,0.08


### Passing stats

URL of passing stats

In [63]:
passing_url = urls_stats[4]

Download HTML of passing page. Table is commented out. Therefore a replace.

In [65]:
players_passing_data = requests.get(passing_url).text.replace('<!--','').replace('-->','')

Wrap in a 'StringIO' object to read from a literal string

In [66]:
players_passing_data = StringIO(players_passing_data)

Create dataframe of players passing stats

In [67]:
players_passing_stats = pd.read_html(players_passing_data, attrs = {'id':'stats_passing'})[0]

Drop the first level of the dataframe

In [68]:
players_passing_stats.columns = players_passing_stats.columns.droplevel()

Drop rows where column names occur because of the 'folded' table

In [69]:
players_passing_stats = players_passing_stats.loc[players_passing_stats['Player'] != 'Player']

Keep only relevant columns

In [70]:
players_passing_stats = players_passing_stats[["Player", "Squad", "Age", "xA", "1/3"]]

Print first rows of dataframe

In [71]:
players_passing_stats.head()

,Player,Squad,Age,xA,1/3
0,Patrick van Aanholt,PSV Eindhoven,32,3.1,68
1,Paxten Aaronson,Vitesse,19,2.0,22
2,Jayden Addai,AZ Alkmaar,17,0.9,4
3,Bobby Adekanye,Go Ahead Eag,24,4.0,27
4,Shawn Adewoye,RKC Waalwijk,23,0.4,73


### Goal and shot creation stats

URL of goal and shot creation stats

In [83]:
gsc_url = urls_stats[6]

Download HTML of goal and shot creation page. Table is commented out. Therefore a replace.

In [84]:
players_gsc_data = requests.get(gsc_url).text.replace('<!--','').replace('-->','')

Wrap in a 'StringIO' object to read from a literal string

In [85]:
players_gsc_data = StringIO(players_gsc_data)

Create dataframe of players goal and shot creation stats

In [86]:
players_gsc_stats = pd.read_html(players_gsc_data, attrs = {'id':'stats_gca'})[0]

Drop the first level of the dataframe

In [87]:
players_gsc_stats.columns = players_gsc_stats.columns.droplevel()

Drop rows where column names occur because of the 'folded' table

In [88]:
players_gsc_stats = players_gsc_stats.loc[players_gsc_stats['Player'] != 'Player']

Keep only relevant columns

In [89]:
players_gsc_stats = players_gsc_stats[["Player", "Squad", "Age", "TO"]]

Drop second column with "TO"

In [95]:
players_gsc_stats = players_gsc_stats.loc[:,~players_gsc_stats.columns.duplicated()]

Drop second "TO" column

Print first rows of dataframe

In [96]:
players_gsc_stats.head()

,Player,Squad,Age,TO
0,Patrick van Aanholt,PSV Eindhoven,32,3
1,Paxten Aaronson,Vitesse,19,3
2,Jayden Addai,AZ Alkmaar,17,0
3,Bobby Adekanye,Go Ahead Eag,24,11
4,Shawn Adewoye,RKC Waalwijk,23,0


### Defense stats

URL of defense stats

In [105]:
defense_url = urls_stats[7]

Download HTML of defense page. Table is commented out. Therefore a replace.

In [106]:
players_defense_data = requests.get(defense_url).text.replace('<!--','').replace('-->','')

Wrap in a 'StringIO' object to read from a literal string

In [107]:
players_defense_data = StringIO(players_defense_data)

Create dataframe of players defense stats

In [108]:
players_defense_stats = pd.read_html(players_defense_data, attrs = {'id':'stats_defense'})[0]

Drop the first level of the dataframe

In [109]:
players_defense_stats.columns = players_defense_stats.columns.droplevel()

Drop rows where column names occur because of the 'folded' table

In [110]:
players_defense_stats = players_defense_stats.loc[players_defense_stats['Player'] != 'Player']

Keep only relevant columns

In [111]:
players_defense_stats = players_defense_stats[["Player", "Squad", "Age", "Att 3rd", "Pass", "Int"]]

Print first rows of dataframe

In [112]:
players_defense_stats.head()

,Player,Squad,Age,Att 3rd,Pass,Int
0,Patrick van Aanholt,PSV Eindhoven,32,3,3,15
1,Paxten Aaronson,Vitesse,19,6,18,9
2,Jayden Addai,AZ Alkmaar,17,2,3,0
3,Bobby Adekanye,Go Ahead Eag,24,3,16,4
4,Shawn Adewoye,RKC Waalwijk,23,4,21,32


## Competitions

In [5]:
# List of competitions with advanced data from Opta for the analysis
list_competitions = ["https://fbref.com/en/comps/9/Premier-League-Stats",
                    "https://fbref.com/en/comps/12/La-Liga-Stats",
                    "https://fbref.com/en/comps/11/Serie-A-Stats",
                    "https://fbref.com/en/comps/20/Bundesliga-Stats",
                    "https://fbref.com/en/comps/13/Ligue-1-Stats",
                    "https://fbref.com/en/comps/10/Championship-Stats",
                    "https://fbref.com/en/comps/23/Eredivisie-Stats",
                    "https://fbref.com/en/comps/22/Major-League-Soccer-Stats",
                    "https://fbref.com/en/comps/32/Primeira-Liga-Stats",
                    "https://fbref.com/en/comps/24/Serie-A-Stats",
                    "https://fbref.com/en/comps/31/Liga-MX-Stats",
                    "https://fbref.com/en/comps/37/Belgian-Pro-League-Stats",
                    "https://fbref.com/en/comps/17/Segunda-Division-Stats",
                    "https://fbref.com/en/comps/60/Ligue-2-Stats",
                    "https://fbref.com/en/comps/33/2-Bundesliga-Stats",
                    "https://fbref.com/en/comps/18/Serie-B-Stats",
                    "https://fbref.com/en/comps/21/Primera-Division-Stats"]

In [6]:
# URL of a competition where stats reside in
competition_url = list_competitions[0]

### Team stats

In [357]:
# Download HTML of standard stats page.
data_teams = requests.get(standard_url)

In [358]:
# Create dataframe of team standard stats of first table on the page. Note: could not solve StringIO error-message here.
team_standard_stats = pd.read_html(data_teams.text)[0]

/var/folders/qq/jf2q9_1s1ln_cgch_rjpr1fm0000gn/T/ipykernel_96527/1474480129.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  team_standard_stats = pd.read_html(data_teams.text)[0]


In [360]:
# Drop the first level of the dataframe
team_standard_stats.columns = team_standard_stats.columns.droplevel()

In [361]:
# Keep only relevant columns
team_standard_stats = team_standard_stats[["Squad", "Age", "MP", "Starts", "Min", "90s"]]

In [362]:
# Specify columns to add a prefix to
cols = team_standard_stats.loc[:, team_standard_stats.columns != "Squad"]

In [363]:
# Add "team_" in front of team variables to distinguish between player stats
team_standard_stats = team_standard_stats.rename(columns = {c: 'team_' + c for c in team_standard_stats.columns if c in cols})

In [377]:
team_standard_stats.head()

,Squad,team_Age,team_MP,team_Starts,team_Min,team_90s
0,Arsenal,25.5,26,286,2340,26.0
1,Aston Villa,27.6,27,297,2430,27.0
2,Bournemouth,26.3,25,275,2250,25.0
3,Brentford,27.4,27,297,2430,27.0
4,Brighton,26.7,27,297,2430,27.0


In [365]:
# Merge both dataframes into one dataframe
merged_standard_stats = player_standard_stats.merge(team_standard_stats
                            [["Squad", "team_Age", "team_MP", "team_Starts", "team_Min", "team_90s"]], 
                                                    on = 'Squad', how = 'left')

In [376]:
merged_standard_stats.head()

,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,90s,team_Age,team_MP,team_Starts,team_Min,team_90s
0,Max Aarons,eng ENG,DF,Bournemouth,24-059,2000,14,12,1085,12.1,26.3,25,275,2250,25.0
1,Bénie Adama Traore,ci CIV,"FW,MF",Sheffield Utd,21-094,2002,8,3,387,4.3,26.6,26,286,2340,26.0
2,Tosin Adarabioyo,eng ENG,DF,Fulham,26-161,1997,13,11,993,11.0,29.3,27,297,2430,27.0
3,Elijah Adebayo,eng ENG,FW,Luton Town,26-056,1998,23,13,1162,12.9,27.4,26,286,2340,26.0
4,Simon Adingra,ci CIV,FW,Brighton,22-062,2002,20,15,1367,15.2,26.7,27,297,2430,27.0


## Shooting stats

### Player stats

In [379]:
# URL of shooting stats
shooting_url = urls_stats[3]

In [380]:
# Download HTML of shooting page. Table is commented out. Therefore a replace.
data_players = requests.get(shooting_url).text.replace('<!--','').replace('-->','')

In [369]:
# Wrap in a 'StringIO' object to read from a literal string
data_players = StringIO(data_players)

In [381]:
# Create dataframe of player shooting stats
player_shooting_stats = pd.read_html(data, attrs = {'id':'stats_shooting'})[0]

In [382]:
# Drop the first level of the dataframe
player_shooting_stats.columns = player_shooting_stats.columns.droplevel()

In [383]:
# Drop rows where column names occur because of the 'folded' table
player_shooting_stats = player_shooting_stats.loc[player_shooting_stats['Player'] != 'Player']

In [ ]:
# Keep only relevant columns
player_shooting_stats = player_shooting_stats[["Player", "Nation", "Pos", "Squad", "Age", "Born", "MP", "Starts",
                                              "Min", "90s"]]

In [384]:
# Show first five rows of dataframe
player_shooting_stats.head()

,Rk,Player,Nation,Pos,Squad,Age,Born,90s,Gls,Sh,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Matches
0,1,Max Aarons,eng ENG,DF,Bournemouth,24-059,2000,12.1,0,1,...,23.9,0,0,0,0.0,0.0,0.02,0.0,0.0,Matches
1,2,Bénie Adama Traore,ci CIV,"FW,MF",Sheffield Utd,21-094,2002,4.3,0,1,...,15.3,0,0,0,0.3,0.3,0.27,-0.3,-0.3,Matches
2,3,Tosin Adarabioyo,eng ENG,DF,Fulham,26-161,1997,11.0,1,9,...,16.2,0,0,0,0.3,0.3,0.04,+0.7,+0.7,Matches
3,4,Elijah Adebayo,eng ENG,FW,Luton Town,26-056,1998,12.9,9,28,...,9.5,0,0,0,5.6,5.6,0.20,+3.4,+3.4,Matches
4,5,Simon Adingra,ci CIV,FW,Brighton,22-062,2002,15.2,6,31,...,15.8,0,0,0,3.0,3.0,0.10,+3.0,+3.0,Matches


### Team stats

In [385]:
# Download HTML of shooting page
data_teams = requests.get(shooting_url)

In [387]:
# Create dataframe of team shooting stats of first table on the page. Note: could not solve StringIO error-message here.
team_shooting_stats = pd.read_html(data_teams.text, match = "Squad Shooting")[0]

/var/folders/qq/jf2q9_1s1ln_cgch_rjpr1fm0000gn/T/ipykernel_96527/1401417894.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  team_shooting_stats = pd.read_html(data_teams.text, match = "Squad Shooting")[0]


In [388]:
# Drop the first level of the dataframe
team_shooting_stats.columns = team_shooting_stats.columns.droplevel()

In [ ]:
# Keep only relevant columns
team_shooting_stats = team_shooting_stats[["Squad", "Age", "MP", "Starts", "Min", "90s"]]

In [389]:
# Show first five rows of dataframe
team_shooting_stats.head()

,Squad,# Pl,90s,Gls,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG
0,Arsenal,25,26.0,60,431,139,32.3,16.58,5.35,0.12,0.37,16.0,3,8,8,53.1,47.0,0.11,6.9,5.0
1,Aston Villa,28,27.0,56,394,146,37.1,14.59,5.41,0.13,0.36,16.2,13,4,4,50.1,47.0,0.12,5.9,5.0
2,Bournemouth,27,25.0,33,333,109,32.7,13.32,4.36,0.10,0.29,16.7,10,1,1,35.8,35.0,0.11,-2.8,-3.0
3,Brentford,28,27.0,38,340,111,32.6,12.59,4.11,0.10,0.32,15.1,12,3,3,43.0,40.8,0.12,-5.0,-5.8
4,Brighton,27,27.0,46,404,157,38.9,14.96,5.81,0.10,0.26,16.5,10,5,5,44.9,41.2,0.10,1.1,-0.2


In [190]:
# Specify columns to add a prefix to
cols = team_shooting_stats.loc[:, team_shooting_stats.columns != "Squad"]

In [191]:
# Add "team_" in front of team variables to distinguish between player stats
team_shooting_stats = team_shooting_stats.rename(columns = {c: 'team_' + c for c in team_shooting_stats.columns if c in cols})

In [ ]:
# Create a loop 
for competition in list_competitions:
    